In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import CreatingNewDataframe
from functions.date_split import split_month_test

%matplotlib inline

from sklearn.model_selection import cross_val_score

np.random.seed(seed=42)

from score_submission import our_scorer, scorer
from sklearn.pipeline import Pipeline

In [2]:
df_initial = pd.read_csv('train.csv.zip', parse_dates=['DATE'])
ids = np.unique(df_initial.ATM_ID)
np.random.shuffle(ids)

In [3]:
import functions.GenerateFis as Features

In [4]:
import functions.GenerateMeansWithOffset as MeansOffset

In [12]:
MeansOffset.CreateColumnsOfMeans(df_initial, window=7)

,mean_minus_7,mean_minus_14,mean_minus_21,mean_minus_28,mean_minus_35
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN


In [11]:
MeansOffset.CreateColumnsOfMeans(df_initial, window=61)

,mean_minus_7,mean_minus_14,mean_minus_21,mean_minus_28,mean_minus_35
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN


In [8]:
df1, df2, df3, df4, df5 = Features.GenerateFis(df_initial)

/home/progiv/idao_final/functions/GenerateFis.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['out_{}_weeks_ago'.format(N)] = df.CLIENT_OUT.shift(7*N)
/home/progiv/idao_final/functions/GenerateFis.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['out_{}_weeks_ago_plus_day'.format(N)] = df.CLIENT_OUT.shift(7*N-1)
/home/progiv/idao_final/functions/GenerateFis.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [10]:
df1

,DATE,ATM_ID,CLIENT_OUT,holiday_dummy,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,...,out_2_weeks_ago_minus_day,out_3_weeks_ago,out_3_weeks_ago_plus_day,out_3_weeks_ago_minus_day,out_4_weeks_ago,out_4_weeks_ago_plus_day,out_4_weeks_ago_minus_day,out_5_weeks_ago,out_5_weeks_ago_plus_day,out_5_weeks_ago_minus_day
287393,2017-08-09,419,18500,0,0,0,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
287394,2017-08-10,419,87300,0,0,0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
287395,2017-08-11,419,47500,0,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
287396,2017-08-12,419,18100,0,0,0,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
287397,2017-08-13,419,5000,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
287398,2017-08-14,419,32300,0,1,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
287399,2017-08-15,419,78100,0,0,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
df1.tail()

,DATE,ATM_ID,CLIENT_OUT,holiday_dummy,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,...,out_2_weeks_ago_minus_day,out_3_weeks_ago,out_3_weeks_ago_plus_day,out_3_weeks_ago_minus_day,out_4_weeks_ago,out_4_weeks_ago_plus_day,out_4_weeks_ago_minus_day,out_5_weeks_ago,out_5_weeks_ago_plus_day,out_5_weeks_ago_minus_day
2,2015-01-03,0,335400,1,0,0,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-01-04,0,379000,1,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-01-05,0,344100,1,1,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2015-01-06,0,575300,1,0,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2015-01-07,0,158200,1,0,0,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_initial.head()

,DATE,ATM_ID,CLIENT_OUT
0,2015-01-01,0,91600
1,2015-01-02,0,136500
2,2015-01-03,0,335400
3,2015-01-04,0,379000
4,2015-01-05,0,344100


In [3]:
from sklearn import datasets, linear_model


In [4]:

from sklearn.base import BaseEstimator, TransformerMixin

In [23]:
class SeparatingAtmIds(BaseEstimator, TransformerMixin):
    """Extract the subject & body from a usenet post in a single pass.

    Takes a sequence of strings and produces a dict of sequences.  Keys are
    `subject` and `body`.
    """
    def fit(self, x, y=None):
        self.train = True
        return self

    def transform(self, X):
        if self.train:
            print("TRAIN ONLY CALLED")
            return CreatingNewDataframe.CleanDataFrame_TRAIN_ONLY(df=X, window_size=15)
        else:
            print("TEST ONLY CALLED")
            return CreatingNewDataframe.CleanDataFrame_TEST(df=X, window_size=15)

In [ ]:
class DroppingAtmIdsIn_X(BaseEstimator, TransformerMixin):
    """Extract the subject & body from a usenet post in a single pass.

    Takes a sequence of strings and produces a dict of sequences.  Keys are
    `subject` and `body`.
    """
    def fit(self, x=None, y=None):
        return self

    def transform(self, X):
        return X 

In [21]:
class FinalStepForSubmission(BaseEstimator, TransformerMixin):
    """
    FINAL
    """
    
    def fit(self, X, y=None):
        return self

    def predict(self, X):
        X = X[['DATE', "ATM_ID", "y_predict"]]
        X.columns = ['DATE', "ATM_ID", "CLIENT_OUT"]
        print(len(np.unique(X.ATM_ID)))
        return X
        
class WrapperForEstimator(BaseEstimator, TransformerMixin):
    """
    FINAL
    """
    def __init__(self, estimator):
        self.estimator = estimator
    
    
    def fit(self, X, y=None):
#         print("fit called")
        self.estimator.fit(X[['ATM_ID']], y)
        return self

    def transform(self, X):
#         print("transform called")
        y_pred = self.estimator.predict(X[['ATM_ID']])
        X_tr = X.copy()
        X_tr['y_predict'] = y_pred
        return X_tr

In [5]:

    
    # Extract the subject & body


    
    # Use FeatureUnion to combine the features from subject and body
#     ('union', FeatureUnion(
#         transformer_list=[

#             # Pipeline for pulling features from the post's subject line
#             ('subject', Pipeline([
#                 ('selector', ItemSelector(key='subject')),
#                 ('tfidf', TfidfVectorizer(min_df=50)),
#             ])),

#             # Pipeline for standard bag-of-words model for body
#             ('body_bow', Pipeline([
#                 ('selector', ItemSelector(key='body')),
#                 ('tfidf', TfidfVectorizer()),
#                 ('best', TruncatedSVD(n_components=50)),
#             ])),

#             # Pipeline for pulling ad hoc features from post's body
#             ('body_stats', Pipeline([
#                 ('selector', ItemSelector(key='body')),
#                 ('stats', TextStats()),  # returns a list of dicts
#                 ('vect', DictVectorizer()),  # list of dicts -> feature matrix
#             ])),

#         ],

#         # weight components in FeatureUnion
#         transformer_weights={
#             'subject': 0.8,
#             'body_bow': 0.5,
#             'body_stats': 1.0,
#         },
#     )),

    # Use a SVC classifier on the combined features
    
pipeline = Pipeline([
    ('SeparatingAtmIds', SeparatingAtmIds()),
    
    ('DroppingAtmIdsIn_X', DroppingAtmIdsIn_X())
    
    ('lasso', WrapperForEstimator(linear_model.Lasso())), #predict step
    
   
    ("submission", FinalStepForSubmission()),
    
    ]
)

NameError: name 'SeparatingAtmIds' is not defined

In [35]:
X = df_initial[['DATE', 'ATM_ID']]
y = df_initial[['CLIENT_OUT']]


In [36]:
pipeline.fit(X, y)

fit called
transform called


Pipeline(memory=None,
     steps=[('lasso', WrapperForEstimator(estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False))), ('submission', FinalStepForSubmission())])

In [37]:
pipeline.predict(X)

transform called
300


,DATE,ATM_ID,CLIENT_OUT
0,2015-01-01,0,315602.973975
1,2015-01-02,0,315602.973975
2,2015-01-03,0,315602.973975
3,2015-01-04,0,315602.973975
4,2015-01-05,0,315602.973975
5,2015-01-06,0,315602.973975
6,2015-01-07,0,315602.973975
7,2015-01-08,0,315602.973975
8,2015-01-09,0,315602.973975
9,2015-01-10,0,315602.973975


In [18]:
X['ATM_ID'].values

array([  0,   0,   0, ..., 419, 419, 419])

In [19]:
current_scores = cross_val_score(pipeline, X, y, scoring=our_scorer,  groups=X['ATM_ID'].values, cv=5)

60
60
60
60
60


In [25]:
current_scores

array([2.72255938, 1.7024016 , 1.7285596 , 1.27134481, 3.40484597])

In [54]:
def CalculateScore(pipeline, df=df_initial):
    test_days = 33
    
    scores = []
    
    len_ids = len(ids)
    cross_val_n = 5
    step_ids = len_ids // 5
    # different time frames with period one month
    for previous_training_days in range(0, 120, 28):
        train_df, test_df = split_month_test(df,
                                             test_days=test_days,
                                             train_days=
                                            )
        current_scores = cross_val_score(lasso, X, y)
        scores.append
        
        
    return scores

In [41]:
np.random.shuffle(ids)

In [43]:
ids = np.array([283, 371, 209,  10, 319, 311, 276, 138,   6, 245, 327,  69, 302,
        55, 257, 305, 403, 291, 203, 231,  92, 142, 343, 345, 127,  52,
       389, 413, 217, 328,  23, 230,  41,  30, 299, 154,   8, 105,  56,
        87, 108,  90, 396,  72,  91,  76, 321, 315, 140, 317, 255, 194,
       330,  89, 415, 385, 113, 107, 272,  31, 318,  71,   7, 262, 243,
        36,  28, 357,  68, 264, 136, 167, 316, 273, 172, 390,  96,  98,
        79, 401, 228, 212,  25, 163,  93, 147, 151,  86,  21,  11, 274,
       119,  81, 309,  47,  22, 252, 199, 380,  80, 314,  82,  38, 258,
       370, 310, 189,  24, 256, 112, 177, 367, 100, 342, 339, 149, 196,
       406, 277, 286,  67, 242, 260, 235, 186, 352,  48, 166, 275, 391,
       182, 141, 238, 150, 383, 376, 213, 246, 249,   3, 148, 193, 248,
       155, 290, 362, 168,  88,  35,  97, 374, 131, 307, 219, 387, 340,
       306,  78, 278,  99, 297, 220,  17,  44,  34, 192, 394, 354, 240,
        63, 111, 288, 341,  51, 103, 338, 181,  32, 407, 191, 280,   0,
       373, 378, 117, 361, 355, 241, 115,  45,  73, 207, 326, 282, 336,
        14, 414, 372,  33, 303,   5, 160,  40, 284, 226, 289, 395, 185,
        74, 180, 169, 404,   9,  84, 369,  77,  54, 386, 216,  50, 161,
       381, 300, 211,  29, 366, 139,  95, 287, 295,  49, 331, 405, 360,
       279,  19,  57, 109, 368, 312, 379, 281, 225,  53, 301, 195, 270,
       304, 347,   4, 128,  65, 178,   2, 176, 126,  60,  94, 285,  43,
       106,  64, 334,  18, 102, 233, 408, 179, 399, 337,  66,  61, 244,
       269, 418, 265, 239,  70,  58, 324, 348,  27, 222, 382, 271, 359,
       205, 173, 208, 116, 101, 298, 159, 419,  26, 267,  85, 129, 375,
       123])

In [47]:
ids

array([283, 371, 209,  10, 319, 311, 276, 138,   6, 245, 327,  69, 302,
        55, 257, 305, 403, 291, 203, 231,  92, 142, 343, 345, 127,  52,
       389, 413, 217, 328,  23, 230,  41,  30, 299, 154,   8, 105,  56,
        87, 108,  90, 396,  72,  91,  76, 321, 315, 140, 317, 255, 194,
       330,  89, 415, 385, 113, 107, 272,  31, 318,  71,   7, 262, 243,
        36,  28, 357,  68, 264, 136, 167, 316, 273, 172, 390,  96,  98,
        79, 401, 228, 212,  25, 163,  93, 147, 151,  86,  21,  11, 274,
       119,  81, 309,  47,  22, 252, 199, 380,  80, 314,  82,  38, 258,
       370, 310, 189,  24, 256, 112, 177, 367, 100, 342, 339, 149, 196,
       406, 277, 286,  67, 242, 260, 235, 186, 352,  48, 166, 275, 391,
       182, 141, 238, 150, 383, 376, 213, 246, 249,   3, 148, 193, 248,
       155, 290, 362, 168,  88,  35,  97, 374, 131, 307, 219, 387, 340,
       306,  78, 278,  99, 297, 220,  17,  44,  34, 192, 394, 354, 240,
        63, 111, 288, 341,  51, 103, 338, 181,  32, 407, 191, 28

In [ ]:
ids[]